In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from scipy.interpolate import interp1d

In [8]:
current_path = Path('.') # current path where the notebook is located
path_data = current_path.joinpath('data')

# Reading the SPHERE transmission table

In [10]:
star_photons_pd = pd.read_csv(path_data.joinpath('star_photons.csv'),index_col=0)
nb_wl = len(star_photons_pd)
dlambda=np.median(np.diff(star_photons_pd.index)) #5 nm
print('There are {0:d} wavelengths from {1:.1f} nm to {2:.1f} nm with a step of {3:.1f} nm'.format(\
    nb_wl,star_photons_pd.index[0],star_photons_pd.index[-1],dlambda))
star_photons_pd.head()
stellar_types = ['A0','G0','M0','M5']
nb_stellar_types = len(stellar_types)
stellar_mag=0
tel_diameter=8 #m
central_obscuration=0.14 # in %
collecting_area=np.pi*tel_diameter**2*(1-central_obscuration**2)/4
Tdust=0.86

There are 427 wavelengths from 500.0 nm to 2630.0 nm with a step of 5.0 nm


In [11]:
star_photons_pd.tail()

,A0,G0,M0,M5,Tsky,Mirror
lambda [nm],,,,,,
2610,0.014545,0.047834,0.321738,5.52960,0.000298,0.979327
2615,0.014508,0.047738,0.320552,5.51170,0.000021,0.979338
2620,0.014481,0.047671,0.319646,5.49821,0.000069,0.979348
2625,0.014465,0.047635,0.319032,5.48933,0.002474,0.979358
2630,0.014460,0.047631,0.318719,5.48526,0.000008,0.979369


In [12]:
zero_points_pd = pd.read_csv(path_data.joinpath('zero_point.csv'),index_col=0)
zero_points_pd.head(20)

,lambdaZP [nm],ZP [ph/m2/s/nm]
Band,,
U,360,75800000.0
B,440,146000000.0
V,550,99900000.0
R,640,73700000.0
I,890,37300000.0
J,1250,19900000.0
H,1650,8890000.0
K,2200,4050000.0


In [14]:
SPHERE_NIR_pd = pd.read_csv(path_data.joinpath('SPHERE_transmission_NIR.csv'),index_col=0)
SPHERE_NIR_pd.head()

,Common path,Visible infrared dichroic BS,Infrared channel cemented doublet (T face irl1),Infrared channel cemented doublet (T face irl2),Infrared ADC (T - prism 1),Infrared ADC (T - prism 2),Infrared ADC (T - prism 3),Infrared ADC (T - prism 4),Infrared channel lens,Infrared DTTS BS,Common IR before DTTS,Total before DTTS,TotalCPIVis,TotalCPIWFS,detectorQE,TotalVisWFS
lambda [nm],,,,,,,,,,,,,,,,
400,0.384032,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.000000,1.000000,0.55,0.000000
450,0.570036,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.777289,1.000000,0.65,0.288004
500,0.607499,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.910020,0.932735,0.76,0.391893
550,0.595509,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.935968,0.957349,0.85,0.453564
600,0.599226,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.923763,0.954534,0.91,0.480821


In [15]:
SPHERE_NIR_pd.tail()

,Common path,Visible infrared dichroic BS,Infrared channel cemented doublet (T face irl1),Infrared channel cemented doublet (T face irl2),Infrared ADC (T - prism 1),Infrared ADC (T - prism 2),Infrared ADC (T - prism 3),Infrared ADC (T - prism 4),Infrared channel lens,Infrared DTTS BS,Common IR before DTTS,Total before DTTS,TotalCPIVis,TotalCPIWFS,detectorQE,TotalVisWFS
lambda [nm],,,,,,,,,,,,,,,,
2100,0.803845,0.976936,0.992659,0.992207,0.9860,0.9835,0.9835,0.9860,0.986239,0.982884,0.904839,0.727350,0.0,1.0,0.0,0.0
2200,0.809734,0.943706,0.988465,0.989284,0.9885,0.9850,0.9850,0.9885,0.991742,0.981368,0.874872,0.708414,0.0,1.0,0.0,0.0
2300,0.810025,0.964956,0.981517,0.982935,0.9855,0.9830,0.9830,0.9855,0.988699,0.983443,0.873676,0.707699,0.0,1.0,0.0,0.0
2400,0.974693,0.937132,0.973419,0.975829,0.9780,0.9800,0.9800,0.9780,0.982849,NaN,0.817719,0.797025,0.0,1.0,0.0,0.0
2500,0.976271,1.000000,NaN,NaN,0.9600,0.9660,0.9660,0.9600,0.970059,NaN,0.859997,0.839590,0.0,1.0,0.0,0.0


In [16]:
interp_fct_SPHERE_NIR = interp1d(SPHERE_NIR_pd.index,SPHERE_NIR_pd['Total before DTTS'],\
                            bounds_error=False,fill_value='extrapolate')
star_photons_pd['SPHERE before DTTS'] = interp_fct_SPHERE_NIR(star_photons_pd.index)

In [17]:
interp_fct_SPHERE_Vis = interp1d(SPHERE_NIR_pd.index,SPHERE_NIR_pd['TotalVisWFS'],\
                            bounds_error=False,fill_value='extrapolate')
star_photons_pd['SPHERE Vis WFS'] = interp_fct_SPHERE_Vis(star_photons_pd.index)

In [18]:
star_photons_pd.head(100)

,A0,G0,M0,M5,Tsky,Mirror,SPHERE before DTTS,SPHERE Vis WFS
lambda [nm],,,,,,,,
500,0.721930,0.556265,0.409828,0.374307,0.854953,0.918934,0.000000,0.391893
505,0.708234,0.556980,0.422488,0.388232,0.856648,0.918621,0.000000,0.398060
510,0.694039,0.557580,0.435352,0.402400,0.861126,0.918307,0.000000,0.404228
515,0.679513,0.558102,0.448351,0.416729,0.864494,0.917993,0.000000,0.410395
520,0.664489,0.558510,0.461555,0.431300,0.866992,0.917679,0.000000,0.416562
...,...,...,...,...,...,...,...,...
975,0.189485,0.423242,1.307330,8.749900,0.945128,0.932822,0.674127,0.045880
980,0.188003,0.419478,1.308960,8.862300,0.959561,0.934418,0.685470,0.037340
985,0.186501,0.415666,1.310540,8.975240,0.972275,0.936003,0.696812,0.028799


In [69]:
star_photons_pd.tail(50)

,A0,G0,M0,M5,Tsky,Mirror,SPHERE before DTTS,SPHERE Vis WFS
lambda [nm],,,,,,,,
2385,0.020228,0.063773,0.486516,8.10388,8.625874e-01,0.978862,0.783626,0.0
2390,0.020097,0.063403,0.482704,8.04429,8.828072e-01,0.978872,0.788092,0.0
2395,0.019964,0.063032,0.478876,7.98444,9.049338e-01,0.978883,0.792559,0.0
2400,0.019831,0.062657,0.475024,7.92421,9.495563e-01,0.978893,0.797025,0.0
2405,0.019697,0.062282,0.471156,7.86373,8.874384e-01,0.978903,0.799153,0.0
2410,0.019563,0.061903,0.467264,7.80286,9.024048e-01,0.978914,0.801282,0.0
2415,0.019428,0.061523,0.463355,7.74174,8.505406e-01,0.978924,0.803410,0.0
2420,0.019291,0.061141,0.459422,7.68022,8.378579e-01,0.978934,0.805538,0.0
2425,0.019155,0.060756,0.455473,7.61846,9.053373e-01,0.978945,0.807666,0.0


In [19]:
def NIR_WFS_photons_per_second(scenario,nb_photons,band, freq):
    """
    Returns the star mag corresponding to the number of photons in the specified band for the 4 stellar types
    Input: 
    - scenario: a dictionnary containing the entries 'lambda min' (in micron), 'lambda max', 'transmission'
    - nb_photons: scalar, the number of photons per frame we would like
    - band: the band (typically H) where we want this minimum number of photons.
    - freq: the frequency in kHz of the AO loop
    Output:
    - a table of limiting magnitudes for each of the 4 stellar types.
    """
    band_wl =  zero_points_pd['lambdaZP [nm]'][band] # wavelength of the ZP 
    nb_photons_mag0 = np.zeros((nb_stellar_types))*np.nan # number of photons collected before 
                                                          # the DTTS for a mag 0 star
    dit_s = 1./(freq*1000.) # DIT in s for one AO frame
    
    for j,stellar_type in enumerate(stellar_types):
        print(j,stellar_type)
        FnormZ = star_photons_pd[stellar_type][band_wl]
        star_flux_mag0_top_atm_ph_per_s = star_photons_pd[stellar_type]/FnormZ*\
                zero_points_pd['ZP [ph/m2/s/nm]'][band]*\
                dlambda*star_photons_pd['Tsky']*collecting_area*np.power(star_photons_pd['Mirror'],3)*Tdust
        star_flux_mag0_before_DTTS_ph_per_s = star_flux_mag0_top_atm_ph_per_s*dit_s*star_photons_pd['SPHERE before DTTS']    
        total_transmission=np.zeros((nb_wl))    
        for i,wl in enumerate(star_photons_pd.index):
            if wl>= scenario['lambda min'] and wl< scenario['lambda max']:
                #print(star_flux_mag0_before_DTTS_ph_per_s)
                total_transmission[i] = star_flux_mag0_before_DTTS_ph_per_s[wl]*scenario['transmission']
        nb_photons_mag0[j] = np.sum(total_transmission)
    print(['{0:3.1e}'.format(nn) for nn in nb_photons_mag0])
    flux_ratio = nb_photons/nb_photons_mag0
    print(['{0:3.1e}'.format(nn) for nn in flux_ratio])
    limit_mag = -2.5*np.log10(flux_ratio)
    return limit_mag

# Testing different scenarios of NIR Py WFS

The scenarios below correspond to the current broad band filter definitions

In [32]:
scenario_Y={'lambda min':1043-140/2,'lambda max':1043+140/2,'transmission':1.}
scenario_J={'lambda min':1245-240/2,'lambda max':1245+240/2,'transmission':1.}
scenario_K={'lambda min':2182-300/2,'lambda max':2182+300/2,'transmission':1.}

print(scenario_Y)
print(scenario_J)
print(scenario_K)

{'lambda min': 973.0, 'lambda max': 1113.0, 'transmission': 1.0}
{'lambda min': 1125.0, 'lambda max': 1365.0, 'transmission': 1.0}
{'lambda min': 2032.0, 'lambda max': 2332.0, 'transmission': 1.0}


We can now see the limiting mag in H in each of those 3 scenarios for 6000 photons at 3kHz

In [36]:
NIR_WFS_photons_per_second(scenario_Y,6000,'H',3)

0 A0
1 G0
2 M0
3 M5
['3.1e+07', '2.3e+07', '1.2e+07', '6.6e+06']
['1.9e-04', '2.6e-04', '5.2e-04', '9.0e-04']


array([9.29756958, 8.96383257, 8.21868708, 7.61116443])

In [37]:
NIR_WFS_photons_per_second(scenario_J,6000,'H',3)

0 A0
1 G0
2 M0
3 M5
['3.4e+07', '2.8e+07', '1.9e+07', '1.5e+07']
['1.8e-04', '2.2e-04', '3.1e-04', '3.9e-04']


array([9.38933004, 9.15660575, 8.76174581, 8.51266955])

In [38]:
NIR_WFS_photons_per_second(scenario_K,6000,'H',3)

0 A0
1 G0
2 M0
3 M5
['1.1e+07', '1.2e+07', '1.3e+07', '1.4e+07']
['5.4e-04', '5.1e-04', '4.5e-04', '4.2e-04']


array([8.16661806, 8.23328053, 8.37420136, 8.43643462])

Below is an extended K band case:

In [28]:
NIR_WFS_photons_per_second({'lambda min':1800,'lambda max':2300,'transmission':1.},6000,'H',3)

0 A0
1 G0
2 M0
3 M5
['1.6e+07', '1.7e+07', '1.9e+07', '1.8e+07']
['3.8e-04', '3.6e-04', '3.1e-04', '3.3e-04']


array([8.5533525 , 8.61377393, 8.76578541, 8.72021247])

Here is a case with a  narrow band K for WFS 

In [39]:
NIR_WFS_photons_per_second({'lambda min':2100,'lambda max':2200,'transmission':1.},6000,'H',3)

0 A0
1 G0
2 M0
3 M5
['4.0e+06', '4.3e+06', '4.9e+06', '5.0e+06']
['1.5e-03', '1.4e-03', '1.2e-03', '1.2e-03']


array([7.06498713, 7.13942775, 7.2853649 , 7.29496223])

Everything below H band sent to Py WFS

In [40]:
NIR_WFS_photons_per_second({'lambda min':950,'lambda max':1480,'transmission':1.},6000,'H',3)

0 A0
1 G0
2 M0
3 M5
['7.9e+07', '6.1e+07', '3.8e+07', '2.8e+07']
['7.6e-05', '9.8e-05', '1.6e-04', '2.2e-04']


array([10.29212725, 10.02660617,  9.51084342,  9.15497954])

Everything below J band sent to Py WFS

In [42]:
NIR_WFS_photons_per_second({'lambda min':950,'lambda max':1125,'transmission':1.},6000,'H',3)

0 A0
1 G0
2 M0
3 M5
['3.8e+07', '2.8e+07', '1.4e+07', '7.9e+06']
['1.6e-04', '2.1e-04', '4.3e-04', '7.6e-04']


array([9.50489607, 9.17371284, 8.41273931, 7.79277778])

Everything below J band sent to Py WFS at 1kHz

In [43]:
NIR_WFS_photons_per_second({'lambda min':950,'lambda max':1125,'transmission':1.},6000,'H',1)

0 A0
1 G0
2 M0
3 M5
['1.1e+08', '8.4e+07', '4.2e+07', '2.4e+07']
['5.3e-05', '7.1e-05', '1.4e-04', '2.5e-04']


array([10.6976992 , 10.36651597,  9.60554245,  8.98558092])

Everything below J band sent to Py WFS at 0.5 kHz

In [44]:
NIR_WFS_photons_per_second({'lambda min':950,'lambda max':1125,'transmission':1.},6000,'H',0.5)

0 A0
1 G0
2 M0
3 M5
['2.3e+08', '1.7e+08', '8.3e+07', '4.7e+07']
['2.6e-05', '3.6e-05', '7.2e-05', '1.3e-04']


array([11.45027419, 11.11909096, 10.35811743,  9.73815591])

everything below J band starting from 850  sent to Py WFS

In [45]:
NIR_WFS_photons_per_second({'lambda min':850,'lambda max':1125,'transmission':1.},6000,'H',3)

0 A0
1 G0
2 M0
3 M5
['4.3e+07', '3.1e+07', '1.5e+07', '8.4e+06']
['1.4e-04', '1.9e-04', '3.9e-04', '7.1e-04']


array([9.62833653, 9.29410397, 8.51254678, 7.87130666])

Now we investigate a case where the transmission is constant (typically 100%) in a range and different in another range (typically 50%)

In [50]:
def NIR_WFS_limit_mag_half_transmission(scenario,nb_photons,band, freq):
    """
    Returns the star mag corresponding to the number of photons in the specified band for the 4 stellar types
    Input: 
    - scenario: a dictionnary containing the entries 'lambda 1' (in micron), 'lambda 2', 
      'lambda 3', 'transmission 12', 'transmission 23'
    - nb_photons: scalar, the number of photons per frame we would like
    - band: the band (typically H) where we want this minimum number of photons.
    - freq: the frequency in kHz of the AO loop
    Output:
    - a table of limiting magnitudes for each of the 4 stellar types.
    """
    band_wl =  zero_points_pd['lambdaZP [nm]'][band] # wavelength of the ZP 
    nb_photons_mag0 = np.zeros((nb_stellar_types))*np.nan # number of photons collected before 
                                                          # the DTTS for a mag 0 star
    dit_s = 1./(freq*1000.) # DIT in s for one AO frame
    
    for j,stellar_type in enumerate(stellar_types):
        print(j,stellar_type)
        FnormZ = star_photons_pd[stellar_type][band_wl]
        star_flux_mag0_top_atm_ph_per_s = star_photons_pd[stellar_type]/FnormZ*\
                zero_points_pd['ZP [ph/m2/s/nm]'][band]*\
                dlambda*star_photons_pd['Tsky']*collecting_area*np.power(star_photons_pd['Mirror'],3)*Tdust
        star_flux_mag0_before_DTTS_ph_per_s = star_flux_mag0_top_atm_ph_per_s*dit_s*star_photons_pd['SPHERE before DTTS']    
        total_transmission=np.zeros((nb_wl))    
        for i,wl in enumerate(star_photons_pd.index):
            if wl>= scenario['lambda 1'] and wl< scenario['lambda 2']:
                #print(star_flux_mag0_before_DTTS_ph_per_s)
                total_transmission[i] = star_flux_mag0_before_DTTS_ph_per_s[wl]*scenario['transmission 12']
            elif wl>= scenario['lambda 2'] and wl< scenario['lambda 3']:
                total_transmission[i] = star_flux_mag0_before_DTTS_ph_per_s[wl]*scenario['transmission 23']
        nb_photons_mag0[j] = np.sum(total_transmission)
    print(['{0:3.1e}'.format(nn) for nn in nb_photons_mag0])
    flux_ratio = nb_photons/nb_photons_mag0
    print(['{0:3.1e}'.format(nn) for nn in flux_ratio])
    limit_mag = -2.5*np.log10(flux_ratio)
    return limit_mag

Full Y band to WFS, 50% J band to WFS

In [47]:
NIR_WFS_limit_mag_half_transmission({'lambda 1':950,'lambda 2':1125,'lambda 3':1480,'transmission 12':1.,\
                                    'transmission 23':0.5},6000,'H',3)

0 A0
1 G0
2 M0
3 M5
['5.8e+07', '4.5e+07', '2.6e+07', '1.8e+07']
['1.0e-04', '1.3e-04', '2.3e-04', '3.4e-04']


array([9.96835082, 9.68183897, 9.09507557, 8.67481458])

Full Y+J band to WFS, 50% H band to WFS

In [52]:
NIR_WFS_limit_mag_half_transmission({'lambda 1':950,'lambda 2':1480,'lambda 3':2000,'transmission 12':1.,\
                                    'transmission 23':0.5},6000,'H',3)

0 A0
1 G0
2 M0
3 M5
['9.4e+07', '7.7e+07', '5.4e+07', '4.2e+07']
['6.4e-05', '7.8e-05', '1.1e-04', '1.4e-04']


array([10.48284822, 10.26885786,  9.88429744,  9.60833866])

Full Y+J band to WFS, 50% H band to WFS at 1kHz

In [53]:
NIR_WFS_limit_mag_half_transmission({'lambda 1':950,'lambda 2':1480,'lambda 3':2000,'transmission 12':1.,\
                                    'transmission 23':0.5},6000,'H',1)

0 A0
1 G0
2 M0
3 M5
['2.8e+08', '2.3e+08', '1.6e+08', '1.3e+08']
['2.1e-05', '2.6e-05', '3.7e-05', '4.8e-05']


array([11.67565136, 11.461661  , 11.07710058, 10.80114179])

# Checking that the model is correct for the current VWFS

In [54]:
def Vis_WFS_photons_per_second(scenario,nb_photons,band, freq):
    """
    Returns the star mag corresponding to the number of photons in the specified band for the 4 stellar types
    Input: 
    - scenario: a dictionnary containing the entries 'lambda min' (in micron), 'lambda max', 'transmission'
    - nb_photons: scalar, the number of photons per frame we would like
    - band: the band (typically H) where we want this minimum number of photons.
    - freq: the frequency in kHz of the AO loop
    Output:
    - a table of limiting magnitudes for each of the 4 stellar types.
    """
    band_wl =  zero_points_pd['lambdaZP [nm]'][band] # wavelength of the ZP 
    nb_photons_mag0 = np.zeros((nb_stellar_types))*np.nan # number of photons collected before 
                                                          # the DTTS for a mag 0 star
    dit_s = 1./(freq*1000.) # DIT in s for one AO frame
    
    for j,stellar_type in enumerate(stellar_types):
        FnormZ = star_photons_pd[stellar_type][band_wl]
        star_flux_mag0_top_atm_ph_per_s = star_photons_pd[stellar_type]/FnormZ*\
                zero_points_pd['ZP [ph/m2/s/nm]'][band]*\
                dlambda*star_photons_pd['Tsky']*collecting_area*np.power(star_photons_pd['Mirror'],3)*Tdust
        star_flux_mag0_before_DTTS_ph_per_s = star_flux_mag0_top_atm_ph_per_s*dit_s*star_photons_pd['SPHERE Vis WFS']    
        total_transmission=np.zeros((nb_wl))    
        for i,wl in enumerate(star_photons_pd.index):
            if wl>= scenario['lambda min'] and wl< scenario['lambda max']:
                #print(star_flux_mag0_before_DTTS_ph_per_s)
                total_transmission[i] = star_flux_mag0_before_DTTS_ph_per_s[wl]*scenario['transmission']
        nb_photons_mag0[j] = np.sum(total_transmission)
    flux_ratio = nb_photons/nb_photons_mag0
    limit_mag = -2.5*np.log10(flux_ratio)
    print(nb_photons_mag0)
    return limit_mag

In [55]:
scenario_R={'lambda min':500,'lambda max':900,'transmission':1.}
Vis_WFS_photons_per_second(scenario_R,6000,'R',1)

[4.00790289e+08 4.21500514e+08 4.94015738e+08 9.89406780e+08]


array([12.06191485, 12.11661714, 12.28897383, 13.04305908])